In [11]:
import pandas as pd

shouldnt I put the teamkills into the thing ? 

In [12]:
game_metadata = pd.read_csv('./out/game_metadata.csv')
game_player_stats = pd.read_csv('./out/game_player_stats.csv')

unique_games = game_metadata['gameid'].size
print(f'Amount of unique games: {unique_games}')
print(50*'-')

print('Types of values in each dataframe:')
print('Game_metadata:')
print(game_metadata.dtypes)

print('')

print('game_player_stats')
print(game_player_stats.dtypes)
print(50*'-')

Amount of unique games: 74105
--------------------------------------------------
Types of values in each dataframe:
Game_metadata:
gameid         object
date           object
league         object
playoffs        int64
patch         float64
gamelength      int64
dtype: object

game_player_stats
gameid                object
playerid              object
teamid                object
side                  object
position              object
champion              object
result                 int64
kills                  int64
deaths                 int64
assists                int64
damagetochampions    float64
visionscore          float64
earnedgold           float64
total cs             float64
golddiffat15         float64
csdiffat15           float64
xpdiffat15           float64
dtype: object
--------------------------------------------------


In [13]:
game_player_stats['teamkills'] = game_player_stats.groupby(['gameid', 'side'])['kills'].transform('sum')
game_player_stats['teamvisionscore'] = game_player_stats.groupby(['gameid', 'side'])['visionscore'].transform('sum')
print(game_player_stats.dtypes)

gameid                object
playerid              object
teamid                object
side                  object
position              object
champion              object
result                 int64
kills                  int64
deaths                 int64
assists                int64
damagetochampions    float64
visionscore          float64
earnedgold           float64
total cs             float64
golddiffat15         float64
csdiffat15           float64
xpdiffat15           float64
teamkills              int64
teamvisionscore      float64
dtype: object


In [14]:
#Create a function that checks if there are decimal numbers
def has_decimal(arr): 
    return (arr % 1 != 0).any()

In [15]:
#Check for decimal numbers
print(has_decimal(game_player_stats['damagetochampions']))
print(has_decimal(game_player_stats['visionscore']))
print(has_decimal(game_player_stats['earnedgold']))
print(has_decimal(game_player_stats['total cs']))
print(has_decimal(game_player_stats['golddiffat15']))
print(has_decimal(game_player_stats['csdiffat15']))
print(has_decimal(game_player_stats['xpdiffat15']))


False
False
False
False
False
False
False


In [22]:
import sys
print(sys.executable)

/opt/anaconda3/bin/python


In [27]:
import dowhy

#Do a test whether there is a causal impact of tvs on team kills

causal_graph = ''' 
digraph {
    teamvisionscore -> teamkills;
    csdiffat15 -> teamvisionscore;
    csdiffat15 -> teamkills;
    golddiffat15 -> teamvisionscore;
    golddiffat15 -> teamkills;
}
'''

model = CausalModel(
    data = game_player_stats,
    treatment = 'teamvisionscore',
    outcome = 'teamkills',
    graph = causal_graph
)

identified_estimand = model.identify_effect()
#print(identified_estimand)
estimate = model.estimate_effect(
    identified_estimand,
    method_name='backdoor.linear_regression'
)
#print(estimate.value)

refutation = model.refute_estimate(
    identified_estimand,
    estimate,
    method_name = 'placebo_treatment_refuter'
)
print(refutation)

/opt/anaconda3/lib/python3.12/site-packages/dowhy/causal_estimators/regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]
/opt/anaconda3/lib/python3.12/site-packages/dowhy/causal_estimators/regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]
/opt/anaconda3/lib/python3.12/site-packages/dowhy/causal_estimators/regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent 

Refute: Use a Placebo Treatment
Estimated effect:0.011748567279594724
New effect:0.0
p value:1.0



/opt/anaconda3/lib/python3.12/site-packages/dowhy/causal_estimators/regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]


outcome : teamkills
treatment: visionscore

graph: 
tvs -> team_kills
csdiffat15 -> tvs
golddiffat15 -> tvs
